## Implementing Grid Search CV on Decision Tree by varying values of max_depth, min_impurity_decrease, changing impurity criterion (gini or entropy)

In [1]:
# after removing data row as index row and changing column names
import pandas as pd
import numpy as np
df=pd.read_csv('C:\\Users\\soham\\Downloads\\adult.data',names=['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income'],header=None)
df 

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [2]:
#replacing "?" by np.nan
df=df.replace(' ?',np.nan)
#replace missing values by mean,mode
for i in df.columns:
    if (type(df[i].dropna().tolist()[0])==str): #we are checking type of first value of each column after dropping NA's from it.
        df[i]=df[i].fillna(df[i].mode()[0])   #So, no problem even if first value of column turns out to be missing
    elif (type(df[i].dropna().tolist()[0])==float):  #we are checking type of first available element of that column using dropna 
        df[i]=df[i].fillna(df[i].mean())
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [3]:
#creating a dataframe with only integer/numerical type columns
dfint=pd.DataFrame()
for i in df.columns:
    if (type(df[i].tolist()[0])==int):
        dfint[i]=df[i].tolist()
dfint

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,39,77516,13,2174,0,40
1,50,83311,13,0,0,13
2,38,215646,9,0,0,40
3,53,234721,7,0,0,40
4,28,338409,13,0,0,40
...,...,...,...,...,...,...
32556,27,257302,12,0,0,38
32557,40,154374,9,0,0,40
32558,58,151910,9,0,0,40
32559,22,201490,9,0,0,20


In [4]:
#creating a dataframe with only integer/numerical type columns
dfobj=pd.DataFrame()
for i in df.columns:
    if (type(df[i].tolist()[0])==str):
        dfobj[i]=df[i].tolist()
dfobj

,workclass,education,marital-status,occupation,relationship,race,sex,native-country,income
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...
32556,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,United-States,<=50K
32557,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,United-States,>50K
32558,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,United-States,<=50K
32559,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,United-States,<=50K


In [5]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
X = dfobj.iloc[:,:-1]
enc.fit(X)

OneHotEncoder(categorical_features=None, categories=None, drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=True)

In [6]:
#converting enc.transform array into a proper dataframe
onehotlabels = enc.transform(X).toarray()

new_columns=[]
for col, values in zip(X.columns, enc.categories_):
    new_columns.extend([col + '_' + str(i) for i in values])

    
df_ohe= pd.concat([dfint,pd.DataFrame(onehotlabels, columns=new_columns)], axis='columns')
df_ohe= df_ohe.rename(columns={"native-country_ Trinadad&Tobago": "native-country_ Trinadad-Tobago"}) #to avoid error in graphviz
df_ohe

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad-Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
0,39,77516,13,2174,0,40,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,50,83311,13,0,0,13,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,38,215646,9,0,0,40,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,53,234721,7,0,0,40,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,28,338409,13,0,0,40,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,257302,12,0,0,38,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32557,40,154374,9,0,0,40,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32558,58,151910,9,0,0,40,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
32559,22,201490,9,0,0,20,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [7]:
df_comb_ohe = pd.concat([df_ohe, df.iloc[:,[-1]]], axis=1)
df_comb_ohe

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,...,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad-Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia,income
0,39,77516,13,2174,0,40,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,<=50K
1,50,83311,13,0,0,13,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,<=50K
2,38,215646,9,0,0,40,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,<=50K
3,53,234721,7,0,0,40,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,<=50K
4,28,338409,13,0,0,40,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,257302,12,0,0,38,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,<=50K
32557,40,154374,9,0,0,40,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,>50K
32558,58,151910,9,0,0,40,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,<=50K
32559,22,201490,9,0,0,20,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,<=50K


In [10]:
impurity_decrease_list = [i for i in np.arange(0.0001,0.000111,0.000002)]
impurity_decrease_list

[0.0001,
 0.000102,
 0.000104,
 0.00010599999999999999,
 0.00010799999999999998,
 0.00010999999999999998]

In [11]:
#Site :- "https://chrisalbon.com/machine_learning/model_selection/model_selection_using_grid_search/"
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Create a pipeline
pipe = Pipeline([('classifier',  DecisionTreeClassifier())])

# Create space of candidate learning algorithms and their hyperparameters
search_space = [{'classifier': [ DecisionTreeClassifier(random_state=0)],
                 'classifier__criterion': ['gini','entropy'],
                 'classifier__max_depth': [i for i in range(35,51)],
                 'classifier__min_impurity_decrease': impurity_decrease_list }]

# Create grid search 
clf = GridSearchCV(pipe, search_space, cv=5, verbose=10)
clf

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('classifier',
                                        DecisionTreeClassifier(class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features=None,
                                                               max_leaf_nodes=None,
                                                               min_impurity_decrease=0.0,
                                                               min_impurity_split=None,
                                                               min_samples_leaf=1,
                                                               min_samples_split=2,
                                                               min_weight_fraction_leaf=0.0,
   

In [12]:
X=df_ohe.iloc[:,:]
y=df.iloc[:,[-1]]

# Fit grid search
clf.fit(X,y)


Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] classifier=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best'), classifier__criterion=gini, classifier__max_depth=35, classifier__min_impurity_decrease=0.0001 
[CV]  classifier=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best'), classifier__criterion=gini, classifier__max_depth=35, classifier_

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


[CV]  classifier=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=35,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0001, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best'), classifier__criterion=gini, classifier__max_depth=35, classifier__min_impurity_decrease=0.0001, score=0.858, total=   0.6s
[CV] classifier=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=35,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0001, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best'), classifier__criterion=gini, classifi

...

[CV]  classifier=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=50,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.000102, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best'), classifier__criterion=entropy, classifier__max_depth=50, classifier__min_impurity_decrease=0.000102, score=0.848, total=   0.6s
[CV] classifier=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=50,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.000102, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best'), classifier__criterion

[CV]  classifier=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=50,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.000104, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=0, splitter='best'), classifier__criterion=entropy, classifier__max_depth=50, classifier__min_impurity_decrease=0.00010599999999999999, score=0.838, total=   0.6s
[CV] classifier=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=50,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.00010599999999999999,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort=False, random_state=0, splitter='b

[CV]  classifier=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=50,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.00010799999999999998,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort=False, random_state=0, splitter='best'), classifier__criterion=entropy, classifier__max_depth=50, classifier__min_impurity_decrease=0.00010799999999999998, score=0.860, total=   0.7s
[CV] classifier=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=50,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.00010799999999999998,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort=False, random_state=

[Parallel(n_jobs=1)]: Done 960 out of 960 | elapsed:  9.5min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('classifier',
                                        DecisionTreeClassifier(class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features=None,
                                                               max_leaf_nodes=None,
                                                               min_impurity_decrease=0.0,
                                                               min_impurity_split=None,
                                                               min_samples_leaf=1,
                                                               min_samples_split=2,
                                                               min_weight_fraction_leaf=0.0,
   

In [13]:
cv_results_=clf.cv_results_
cv_results_

{'mean_fit_time': array([0.60917077, 0.45414023, 0.52449994, 0.54408822, 0.5260983 ,
        0.44434547, 0.47232947, 0.51990256, 0.44134698, 0.44094672,
        0.41916142, 0.44194598, 0.43635101, 0.41356778, 0.42775445,
        0.44614496, 0.47452846, 0.49251895, 0.4847229 , 0.49051919,
        0.46593342, 0.46393385, 0.43675013, 0.46633425, 0.43895264,
        0.50711045, 0.49851875, 0.50970793, 0.43395133, 0.45454025,
        0.48652058, 0.46773281, 0.51710553, 0.49171796, 0.47173014,
        0.49811459, 0.47652774, 0.46053591, 0.44194689, 0.46033597,
        0.45254054, 0.44234657, 0.47592721, 0.50231552, 0.54668746,
        0.5716733 , 0.52929683, 0.50271244, 0.51270576, 0.51350594,
        0.49891434, 0.51110797, 0.52410069, 0.498912  , 0.48612161,
        0.51750469, 0.49691563, 0.45613961, 0.48332238, 0.46813159,
        0.47832565, 0.51170645, 0.49611583, 0.50431075, 0.46993051,
        0.52569852, 0.44814396, 0.48172374, 0.46273513, 0.46393471,
        0.45473814, 0.46373453,

In [14]:
dfcv = pd.DataFrame(data=cv_results_)
dfcv

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__criterion,param_classifier__max_depth,param_classifier__min_impurity_decrease,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.609171,0.127863,0.045378,0.015043,"DecisionTreeClassifier(class_weight=None, crit...",gini,35,0.0001,{'classifier': DecisionTreeClassifier(class_we...,0.850453,0.858415,0.861794,0.866708,0.857647,0.859003,0.005337,33
1,0.454140,0.024704,0.036979,0.005828,"DecisionTreeClassifier(class_weight=None, crit...",gini,35,0.000102,{'classifier': DecisionTreeClassifier(class_we...,0.850299,0.857647,0.861026,0.866861,0.857801,0.858727,0.005376,81
2,0.524500,0.033479,0.046375,0.004629,"DecisionTreeClassifier(class_weight=None, crit...",gini,35,0.000104,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.857647,0.861026,0.866400,0.858415,0.858911,0.004979,65
3,0.544088,0.050538,0.040577,0.004027,"DecisionTreeClassifier(class_weight=None, crit...",gini,35,0.000106,{'classifier': DecisionTreeClassifier(class_we...,0.850914,0.857801,0.860872,0.866400,0.859029,0.859003,0.005002,33
4,0.526098,0.042387,0.044574,0.003381,"DecisionTreeClassifier(class_weight=None, crit...",gini,35,0.000108,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.858108,0.861179,0.866554,0.859029,0.859187,0.005007,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.600458,0.012664,0.038776,0.004788,"DecisionTreeClassifier(class_weight=None, crit...",entropy,50,0.000102,{'classifier': DecisionTreeClassifier(class_we...,0.837709,0.840602,0.848127,0.857494,0.847359,0.846258,0.006871,163
188,0.613047,0.017498,0.034579,0.001852,"DecisionTreeClassifier(class_weight=None, crit...",entropy,50,0.000104,{'classifier': DecisionTreeClassifier(class_we...,0.837095,0.838759,0.848894,0.859183,0.849509,0.846688,0.008048,147
189,0.618246,0.024191,0.034180,0.003058,"DecisionTreeClassifier(class_weight=None, crit...",entropy,50,0.000106,{'classifier': DecisionTreeClassifier(class_we...,0.837863,0.839373,0.849048,0.857955,0.849816,0.846811,0.007397,131
190,0.612850,0.028065,0.033182,0.005488,"DecisionTreeClassifier(class_weight=None, crit...",entropy,50,0.000108,{'classifier': DecisionTreeClassifier(class_we...,0.838477,0.839988,0.849662,0.860258,0.850430,0.847763,0.007919,115


In [25]:
#to find models which ranked 1st
dfcv[dfcv['rank_test_score']==1]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__criterion,param_classifier__max_depth,param_classifier__min_impurity_decrease,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,0.444345,0.051244,0.038978,0.006384,"DecisionTreeClassifier(class_weight=None, crit...",gini,35,0.00011,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.860104,0.861333,0.866708,0.859029,0.859648,0.005038,1
11,0.441946,0.020200,0.036180,0.009103,"DecisionTreeClassifier(class_weight=None, crit...",gini,36,0.00011,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.860104,0.861333,0.866708,0.859029,0.859648,0.005038,1
17,0.492519,0.028305,0.040177,0.009233,"DecisionTreeClassifier(class_weight=None, crit...",gini,37,0.00011,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.860104,0.861333,0.866708,0.859029,0.859648,0.005038,1
23,0.466334,0.029258,0.036179,0.003866,"DecisionTreeClassifier(class_weight=None, crit...",gini,38,0.00011,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.860104,0.861333,0.866708,0.859029,0.859648,0.005038,1
29,0.454540,0.020796,0.032781,0.003058,"DecisionTreeClassifier(class_weight=None, crit...",gini,39,0.00011,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.860104,0.861333,0.866708,0.859029,0.859648,0.005038,1
35,0.498115,0.031463,0.035980,0.005618,"DecisionTreeClassifier(class_weight=None, crit...",gini,40,0.00011,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.860104,0.861333,0.866708,0.859029,0.859648,0.005038,1
41,0.442347,0.020903,0.035379,0.002652,"DecisionTreeClassifier(class_weight=None, crit...",gini,41,0.00011,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.860104,0.861333,0.866708,0.859029,0.859648,0.005038,1
47,0.502712,0.031832,0.042175,0.002399,"DecisionTreeClassifier(class_weight=None, crit...",gini,42,0.00011,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.860104,0.861333,0.866708,0.859029,0.859648,0.005038,1
53,0.498912,0.021411,0.038776,0.004954,"DecisionTreeClassifier(class_weight=None, crit...",gini,43,0.00011,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.860104,0.861333,0.866708,0.859029,0.859648,0.005038,1
59,0.468132,0.024240,0.040177,0.005525,"DecisionTreeClassifier(class_weight=None, crit...",gini,44,0.00011,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.860104,0.861333,0.866708,0.859029,0.859648,0.005038,1


## As we can see above, total 16 models ranked 1st with the same accuracy

In [18]:
#to find models which ranked in top 20
dfcv[dfcv['rank_test_score'].between(0,20, inclusive=True)]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__criterion,param_classifier__max_depth,param_classifier__min_impurity_decrease,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
4,0.526098,0.042387,0.044574,0.003381,"DecisionTreeClassifier(class_weight=None, crit...",gini,35,0.000108,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.858108,0.861179,0.866554,0.859029,0.859187,0.005007,17
5,0.444345,0.051244,0.038978,0.006384,"DecisionTreeClassifier(class_weight=None, crit...",gini,35,0.00011,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.860104,0.861333,0.866708,0.859029,0.859648,0.005038,1
10,0.419161,0.021262,0.033980,0.005545,"DecisionTreeClassifier(class_weight=None, crit...",gini,36,0.000108,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.858108,0.861179,0.866554,0.859029,0.859187,0.005007,17
11,0.441946,0.020200,0.036180,0.009103,"DecisionTreeClassifier(class_weight=None, crit...",gini,36,0.00011,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.860104,0.861333,0.866708,0.859029,0.859648,0.005038,1
16,0.474528,0.040313,0.039977,0.008119,"DecisionTreeClassifier(class_weight=None, crit...",gini,37,0.000108,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.858108,0.861179,0.866554,0.859029,0.859187,0.005007,17
17,0.492519,0.028305,0.040177,0.009233,"DecisionTreeClassifier(class_weight=None, crit...",gini,37,0.00011,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.860104,0.861333,0.866708,0.859029,0.859648,0.005038,1
22,0.436750,0.013122,0.035981,0.003687,"DecisionTreeClassifier(class_weight=None, crit...",gini,38,0.000108,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.858108,0.861179,0.866554,0.859029,0.859187,0.005007,17
23,0.466334,0.029258,0.036179,0.003866,"DecisionTreeClassifier(class_weight=None, crit...",gini,38,0.00011,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.860104,0.861333,0.866708,0.859029,0.859648,0.005038,1
28,0.433951,0.016231,0.033780,0.005342,"DecisionTreeClassifier(class_weight=None, crit...",gini,39,0.000108,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.858108,0.861179,0.866554,0.859029,0.859187,0.005007,17
29,0.454540,0.020796,0.032781,0.003058,"DecisionTreeClassifier(class_weight=None, crit...",gini,39,0.00011,{'classifier': DecisionTreeClassifier(class_we...,0.851067,0.860104,0.861333,0.866708,0.859029,0.859648,0.005038,1
